In [1]:
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np
from pathlib import Path
import random
import plotly.express as px
from scipy import ndimage
import math
from tqdm.auto import tqdm
import pandas as pd
from skimage.filters import difference_of_gaussians

In [2]:
import sys
sys.path.append('../../30_data_tools/')

from helper import load_dotenv

In [3]:
dotenv = load_dotenv()

In [6]:
tile_dataset = dotenv['TILE_DATASET_DIR'] / 'train'

In [7]:
moire_images = list( (tile_dataset / 'moire').glob('./*.jpg') )

In [8]:
def get_fft( input_img ):
    ft = np.fft.ifftshift(np.array(input_img)[:,:,0])
    ft = np.fft.fft2(ft)
    ft = np.fft.fftshift(ft)
    
    return ft

def invert_fft (input_real, input_imaginary):
    input = (input_real + 1j * input_imaginary)
    ift = np.fft.ifftshift(input)
    ift = np.fft.ifft2(ift)
    ift = np.fft.fftshift(ift)
    return ift.real

In [9]:
def get_spherical_coordinate( x, y ):
    r = math.sqrt( x ** 2 + y ** 2 )
    angle = math.degrees(math.atan2(y,x))

    return r, angle

In [10]:
def get_carthesian_coordinate( r, angle ):
    x = r * math.cos(math.radians(angle))
    y = r * math.sin(math.radians(angle))
    return x, y

In [11]:
def limit_frequencies( fft, inner_limit=None, outer_limit=None ):
    center = (fft.shape[1] / 2, fft.shape[0] / 2)
    for y in range(fft.shape[0]):
        for x in range(fft.shape[1]):
            r = math.sqrt( abs(center[0] - x) ** 2 + abs(center[1] - y) ** 2 )
            
            if outer_limit is not None and r > outer_limit:
                fft[y,x] = 0
    
            if inner_limit is not None and r < inner_limit:
                fft[y,x] = 0

    return fft

In [ ]:
moire_path = random.choices(moire_images, k=1)[0]
non_moire_path = tile_dataset / 'no_moire' / moire_path.name

moire_img = Image.open( moire_path )
no_moire_img = Image.open( non_moire_path )

diff_img = Image.fromarray(np.array(moire_img) - np.array(no_moire_img))
diff_img = diff_img.filter(ImageFilter.BoxBlur(2))
enhancer = ImageEnhance.Contrast(diff_img)
diff_img = enhancer.enhance(2)


fft = get_fft(diff_img)
fft = limit_frequencies( fft, inner_limit=10, outer_limit=140 )

display_image = Image.new(size=(moire_img.size[0]*3, moire_img.size[1]), mode='L')
display_image.paste(moire_img, (0,0))
display_image.paste(no_moire_img, (moire_img.size[0],0))
display_image.paste(diff_img, (moire_img.size[0]*2,0))

display_image

In [ ]:
filtered = ndimage.uniform_filter( np.abs(get_fft(no_moire_img)) , size=3)

In [ ]:
filtered[135:145,135:145] = 0

In [ ]:
px.imshow( filtered )

In [ ]:
filtered = ndimage.uniform_filter( np.abs(fft) , size=3)#[85:195,85:195]
#filtered = filtered ** 2
#filtered[filtered < filtered.max() * 0.6] = 0
#filtered = ndimage.uniform_filter( np.abs(filtered) , size=5)

px.imshow( filtered )

In [ ]:
def get_radiant_map( frequency_img ):
    out_map = np.zeros((
        round(math.sqrt((frequency_img.shape[1] / 2) ** 2 + (frequency_img.shape[0] / 2) ** 2)) + 1, # max. Radius (45°)
        360 # 360°
    ))
    
    for y_raw in range(frequency_img.shape[0]):
        for x_raw in range(frequency_img.shape[1]):
            x = x_raw - frequency_img.shape[1] / 2
            y = y_raw - frequency_img.shape[0] / 2
    
            r, angle = get_spherical_coordinate( x, y )
            
            out_map[
                round(r),
                round(angle) + 180 - 1
            ] += frequency_img[y_raw,x_raw]

    return out_map

In [ ]:
px.imshow(
    difference_of_gaussians( filtered, 1.5 )
)

In [ ]:
def get_main_impulses( input, relevant_level=0.66 ):
    y_positions, x_positions = np.where(input >= input.max() * relevant_level)
    main_impulses = []

    for i in range(y_positions.shape[0]):
            x = x_positions[i] - input.shape[1] / 2
            y = y_positions[i] - input.shape[0] / 2
            
            r, angle = get_spherical_coordinate(x,y)
            #angle += 90 # Anpassung um 0° in der vertikalen, nicht der horizontalen zu haben
            intensity = input[y_positions[i],x_positions[i]]
        
            main_impulses.append((r, angle, intensity))

    return main_impulses

In [ ]:
# anzahl der Hauptimpulse
# Clusteranalyse
def get_clusters_of_impulses( impulses, max_distance=5 ):
    clusters = []
    sorted_impulses = [False for imp in impulses]
    
    def sort_impulse( imp_index ):
        imp = impulses[imp_index]
        connected_impulses = [imp]
        sorted_impulses[imp_index] = True
    
        for i in range(len(impulses)):
            if sorted_impulses[i] == False:
                imp2 = impulses[i]
                distance = math.sqrt(
                    abs(imp[0] - imp2[0]) ** 2 +
                    abs(imp[1] - imp2[1]) ** 2
                )
    
                if distance <= max_distance:
                    connected_impulses += sort_impulse( i )
    
        return connected_impulses
    
    
    while False in sorted_impulses:
        clusters.append( sort_impulse(sorted_impulses.index(False)) )

    return clusters

In [ ]:
def get_clusters_of_impulses( impulses, max_distance=5 ):
    clusters = []
    sorted_impulses = [False for imp in impulses]
    
    while False in sorted_impulses:
        start_idx = sorted_impulses.index(False)
        cluster = []
        clusters.append(cluster)
        queue = [start_idx]
        sorted_impulses[start_idx] = True

        while len(queue) > 0:
            impulse_idx = queue.pop()
            imp = impulses[impulse_idx]
            cluster.append(imp)

            for i in range(len(impulses)):
                if sorted_impulses[i] == False:
                    imp2 = impulses[i]
                    distance = math.sqrt(
                        abs(imp[0] - imp2[0]) ** 2 +
                        abs(imp[1] - imp2[1]) ** 2
                    )
        
                    if distance <= max_distance:
                        queue.append(i)
                        sorted_impulses[i] = True

    return clusters

In [ ]:
def get_processed_diff_img( moire_path ):
    non_moire_path = tile_dataset / 'no_moire' / moire_path.name

    moire_img = Image.open( moire_path )
    no_moire_img = Image.open( non_moire_path )
    
    diff_img = Image.fromarray(np.array(moire_img) - np.array(no_moire_img))
    diff_img = diff_img.filter(ImageFilter.BoxBlur(2))
    enhancer = ImageEnhance.Contrast(diff_img)
    diff_img = enhancer.enhance(2)

    return diff_img

In [ ]:
def normalize_impulses( impulses ):
    normalized_impulses = {}
    
    for r, angle, intensity in impulses:
        r = round(r)
        angle = round(angle)
    
        if angle >= 0 and angle < 180:
            key = (r,angle)
    
            if key not in normalized_impulses:
                normalized_impulses[key] = []
    
            normalized_impulses[key].append(intensity)
    
    return [
        (key[0], key[1], sum(normalized_impulses[key]) / len(normalized_impulses[key]))
        for key in normalized_impulses
    ]

In [ ]:
0 / 0

# Impulse herausschnippeln

In [ ]:
display_image = Image.new(size=(moire_img.size[0]*3, moire_img.size[1]), mode='L')
display_image.paste(moire_img, (0,0))
display_image.paste(no_moire_img, (moire_img.size[0],0))
display_image.paste(diff_img, (moire_img.size[0]*2,0))

display_image

In [ ]:
def get_impulse_rows_of_tile( moire_path ):
    diff_img = get_processed_diff_img( moire_path )
    
    fft = get_fft(diff_img)
    fft = limit_frequencies( fft, inner_limit=10, outer_limit=140 )
    filtered = ndimage.uniform_filter( np.abs(fft) , size=3)[42:182,45:182]
    dog_fft = difference_of_gaussians( filtered, 1.5 )
    
    # impulse normalisieren
    all_impulses = get_main_impulses( dog_fft )
    normalized_impulses = normalize_impulses( all_impulses )
    clusters = get_clusters_of_impulses( normalized_impulses )
    
    rows = []
    for i in range(len(clusters)):
        c = clusters[i]
        radius = np.array([imp[0] for imp in c])
        degrees = np.array([imp[1] for imp in c])
        impulses = np.array([imp[2] for imp in c])
        
        rows.append({
            'img_name' : moire_path.name,
            'impulse_idx' : i,
            'radius_mean' : radius.mean(),
            'radius_std' : radius.std(),
            'radius_min' : radius.min(),
            'radius_max' : radius.max(),
            'radius_span' : radius.max() - radius.min(),
            'degree_mean' : degrees.mean(),
            'degree_std' : degrees.std(),
            'degree_min' : degrees.min(),
            'degree_max' : degrees.max(),
            'degree_span' : degrees.max() - degrees.min(),
            'impulse_mean' : impulses.mean(),
            'impulse_std' : impulses.std(),
            'impulse_min' : impulses.min(),
            'impulse_max' : impulses.max(),
            'impulse_span' : impulses.max() - impulses.min(),
        })

    return rows

In [ ]:
get_impulse_rows_of_tile( moire_path )

In [ ]:
rows = []

for moire_path in tqdm(moire_images):
    rows += get_impulse_rows_of_tile( moire_path )

In [ ]:
df = pd.DataFrame.from_dict(rows)

In [ ]:
df.groupby('img_name').count().impulse_idx.describe()

In [ ]:
df.loc[
    df.img_name == 'Eqj8Cs7jK0.0000.jpg'
]

In [ ]:
moire_path = list(tile_dataset.glob('./**/moire/Nn_sYp1FWI.0142.jpg'))[0]
diff_img = get_processed_diff_img( moire_path )

fft = get_fft(diff_img)
fft = limit_frequencies( fft, inner_limit=15, outer_limit=140 )
filtered = ndimage.uniform_filter( np.abs(fft) , size=3)[42:182,42:182]
dog_fft = difference_of_gaussians( filtered, 1.5 )

In [ ]:
pd.DataFrame.from_dict(get_impulse_rows_of_tile( moire_path ))

In [ ]:
74,41

In [ ]:
math.sqrt( 4 ** 2 + 29 ** 2 )

In [ ]:
px.imshow( dog_fft )

In [ ]:
import cv2
from sklearn.cluster import KMeans

In [ ]:
X = []

for moire_path in tqdm(selected_moires):
    non_moire_path = tile_dataset / 'no_moire' / moire_path.name
    
    moire_img = Image.open( moire_path )
    no_moire_img = Image.open( non_moire_path )
    
    diff_img = Image.fromarray(np.array(moire_img) - np.array(no_moire_img))
    diff_img = diff_img.filter(ImageFilter.BoxBlur(2))
    enhancer = ImageEnhance.Contrast(diff_img)
    diff_img = enhancer.enhance(2)
    
    fft = get_fft(diff_img.filter(ImageFilter.BoxBlur(2)))
    fft = limit_frequencies( fft, inner_limit=10, outer_limit=130 )

    filtered = ndimage.uniform_filter( np.abs(fft) , size=5)[85:195,85:195]
    #filtered = filtered ** 2
    filtered[filtered < filtered.max() * 0.6] = 0
    filtered = ndimage.uniform_filter( np.abs(filtered) , size=5)
                
    resized = cv2.resize(
        filtered,
        (50,50)
    )
    resized /= resized.max()

    X.append(resized)

In [ ]:
X_train = np.array([entry.reshape((50*50,)) for entry in X])

In [ ]:
def display_image_grid( images, img_size=(280,280) ):
    cols = 5
    rows = math.ceil( len(images) / cols )
    
    display_image = Image.new(
        size=(img_size[0]*cols,img_size[1]*rows),
        mode='RGB'
    )
    
    for i in range(len(images)):
        row = math.floor(i / cols)
        col = i % cols

        display_image.paste(
            Image.open(images[i]),
            (col*img_size[0],row*img_size[1])
        )

    return display_image

# Silhouette Analysis

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
for n_clusters in range(2,10):

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(X_train)
    cluster_labels = clusterer.fit_predict(X_train)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X_train, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X_train, cluster_labels)

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0, n_init="auto").fit(X_train)

In [ ]:
display_image_grid( [selected_moires[i] for i in range(len(selected_moires)) if kmeans.labels_[i] == 2][:50] )

In [ ]:
moire_path = random.choice(moire_images)
non_moire_path = tile_dataset / 'no_moire' / moire_path.name

moire_img = Image.open( moire_path )
no_moire_img = Image.open( non_moire_path )

diff_img = Image.fromarray(np.array(moire_img) - np.array(no_moire_img))

fft = get_fft(diff_img.filter(ImageFilter.BoxBlur(2)))
fft = limit_frequencies( fft, inner_limit=10, outer_limit=130 )

r, angle = get_main_impulse( fft )
angle = angle + 90
T = 1 / r * 280

print(T, angle)

px.imshow(ndimage.uniform_filter( np.abs(fft) , size=5))

In [ ]:
coors = [
    (155,117),
    (123,145),
    (137,155),
    (129,123),
    (147,113),
    (142,120),
    (128,128),
    (129,145),
    (154,150),
    (150,143),
    (117,138)
]

In [ ]:
for co in coors:
    print( get_spherical_coordinate(140 - co[0],140 - co[1]) )

In [ ]:
moire_img

In [ ]:
moire_path

In [ ]:
diff_img.save('./test.jpg')